In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from Thinet import Thinet
from DataManager import load_data_from_file

In [2]:
data = load_data_from_file(r'./data/cifar10_features_5000.npz')

Conv1 weights:  (5, 5, 3, 64)
Conv2 input:  (5000, 12, 12, 64)
Conv2 weights:  (5, 5, 64, 64)
Conv2 output:  (5000, 12, 12, 64)
Loaded 5000 samples


Compute compression

In [ ]:
input_map  = data['conv2_input']    # (5000,12,12,64)
weights    = data['conv2_weights']  # (5,5,3,64)
output_map = data['conv2_output']   # (5000,12,12,64)
rates      = [0.75, 0.5, 0.375, 0.25, 0.125, 0.0625]
num_points = 50

for rate in rates:
    thinet = Thinet(input_map, weights, output_map, rate)

    thinet.compress(num_points)

    output_file = r'./data/compression/thinet_compression_%d.npy' % (rate * 64)
    thinet.save_data_to_file(output_file)
    print("Saved to: %s" % output_file)

In [ ]:
rate = 0.5
data_dict = np.load(r'./data/compression/thinet_compression_%d.npy' % (rate * 64)).item()
print("Filters to keep: ", data_dict['keep_filters'])
print("Filters to prune: ", data_dict['remove_filters'])
print(data_dict['weights_wo_reconst'].shape)
print(data_dict['weights_reconst'].shape)

Apply compression

In [3]:
# Just some dummy data for Thinet creation
input_map  = data['conv2_input']    # (5000,12,12,64)
weights    = data['conv2_weights']  # (5,5,3,64)
output_map = data['conv2_output']   # (5000,12,12,64)
rates      = [0.75, 0.5, 0.375, 0.25, 0.125, 0.0625]

model_64_file = r'./data/models/model_64.npy'
for rate in rates:
    compression_data_file = r'./data/compression/thinet_compression_%d.npy' % (rate * 64)
    
    # Without reconstruction
    model_compressed_file = r'./data/models/model_compressed_%d.npy' % (rate * 64)
    thinet = Thinet(input_map, weights, output_map, rate)
    thinet.load_data_from_file(compression_data_file)
    thinet.update_model(model_64_file, model_compressed_file, False)
    
    # With reconstruction
    model_compressed_file = r'./data/models/model_compressed_reconst_%d.npy' % (rate * 64)
    thinet = Thinet(input_map, weights, output_map, rate)
    thinet.load_data_from_file(compression_data_file)
    thinet.update_model(model_64_file, model_compressed_file, True)

Keep filters: 48 : 
Remove filters: 16 : 
Weights wo reconstruction:  (5, 5, 48, 64)
Weights with reconstruction:  (5, 5, 48, 64)

Removed parameters:

	[conv1][weights] (5, 5, 3, 64) --> (5, 5, 3, 48)
	[conv1][biases] (64,) --> (48,)

Updated parameters:

	[conv2][weights] (5, 5, 64, 64) --> (5, 5, 48, 64)

Keep filters: 48 : 
Remove filters: 16 : 
Weights wo reconstruction:  (5, 5, 48, 64)
Weights with reconstruction:  (5, 5, 48, 64)

Removed parameters:

	[conv1][weights] (5, 5, 3, 64) --> (5, 5, 3, 48)
	[conv1][biases] (64,) --> (48,)

Updated parameters:

	[conv2][weights] (5, 5, 64, 64) --> (5, 5, 48, 64)

Keep filters: 32 : 
Remove filters: 32 : 
Weights wo reconstruction:  (5, 5, 32, 64)
Weights with reconstruction:  (5, 5, 32, 64)

Removed parameters:

	[conv1][weights] (5, 5, 3, 64) --> (5, 5, 3, 32)
	[conv1][biases] (64,) --> (32,)

Updated parameters:

	[conv2][weights] (5, 5, 64, 64) --> (5, 5, 32, 64)

Keep filters: 32 : 
Remove filters: 32 : 
Weights wo reconstruction:  